
# **Laboratorium 002: Skuteczne promptowanie modeli LLM w Ollama**

## **Cel zajęć**

- Wyćwiczenie praktycznych technik **prompt engineeringu** na lokalnych modelach uruchamianych przez **Ollama**.  
- Zastosowanie **struktur i ról** (system/developer, user, assistant) w praktyce – z naciskiem na *kompozycję promptów*, a nie pojedyncze pytania.  
- Przeprowadzenie **ewaluacji promptów** (PromptFoo) i analizy zachowania modeli.  
- Zrozumienie zasad **bezpieczeństwa, prompt injection, anonimizacji** i dobrych praktyk projektowych.

---

## **1. Przygotowanie środowiska i prerekwizyty**

- Zainstalowana **Ollama** i dwa modele w konwencji *polski – niepolski* (`llama3.1:8b`, `gemma2:2b`, `SpeakLeash/bielik-*`).  

Test działania modelu:

```bash
echo "Napisz rymowankę o najlepszym wykładowcy na WI." | ollama run gemma2:2b
```
---

- Możemy też przenieść się z pracą do notebooka, wtedy nasz warsztat będzie wyglądał tak:

In [2]:
import ollama

response = ollama.chat(
    model='gemma2:2b',
    messages=[{'role': 'user', 'content': 'Napisz rymowankę o najlepszym wykładowcy na WI'}]
)
print(response['message']['content'])

W Wi, gdzie wiedza się rozpala,
Wykładowca geniusz w swoich ram,
Uczy z pasją i tak samo wylewa,
Książki i historie, bez lęku i strachu.

Zawsze uśmiechnięty, błyskotliwy,
Rozwijamy wiedzę, co do tego nie ma wątpliwości,
W jego słowach piękno, lekkość i radość,
Wspaniały wykładowca, nasz wielki szacunek. 





## **2. Struktura i techniki promptowania**

### 2.1 Zero-shot

**Zero‑shot prompting** to sposób formułowania zapytania do dużego modelu językowego, w którym nie podajemy mu ani jednego przykładu wykonanego zadania. Model dostaje wyłącznie instrukcję („zrób X”) i całą resztę dedukuje sam na podstawie wiedzy nabytej w treningu.

Zadania, do których używa się **zero-shot prompting**:
1. Szybkie prototypowanie – gdy chcemy sprawdzić, czy model może poradzić sobie z danym zadaniem (czy jego architektura i materiał treningowy na to pozwalają).

2. Zadania ogólne o prostym formacie wyjściowym (tłumaczenia, parafrazy, proste klasyfikacje).

3. Systemy low-code/no-code – użytkownik pisze naturalnym językiem, a model odpowiada bez konfiguracji.

4. Ekstrakcja informacji _ad hoc_ (imię, mail, daty z tekstu itp.) tam, gdzie nie opłaca się trenować lub fine-tune'ować modelu.


Kiedy **zero-shot** jest najbardziej efektywne?
- brak lub szczątkowe dane oznaczone – nie mamy przykładowych par wejście→wyjście.
- zadanie jest podobne do scenariuszy widzianych w treningu (np. „podsumuj”, „przetłumacz”, „odpowiedz TAK/NIE”).
- liczy się czas wdrożenia – chcemy odpowiedzi „tu i teraz”, bez skrupulatnego modyfikowania promptu.

Kiedy nie wystarcza?
- złożone transformacje wymagające ścisłej struktury lub niestandardowych reguł.
- zadania domenowe z mało popularnym żargonem (medycyna, prawo) – wtedy lepszy jest few-shot lub RAG.

```bash
echo "Wypisz 5 słów kluczowych z notatki o spotkaniu Koła Naukowego BIT." | ollama run gemma2:2b
```

🔵 Spotkanie Koła Naukowego BIT

📅 Środa, godz. 14:00
📍 Sala 4.30D, Wydział Informatyki

Serdecznie zapraszamy wszystkich studentów — zarówno obecnych członków, jak i osoby, które dopiero chcą dołączyć — na kolejne spotkanie Koła Naukowego BIT!

💡 Plan spotkania:

- Omówienie planu projektów na semestr zimowy – propozycje tematów z zakresu AI, bezpieczeństwa i tworzenia aplikacji webowych.

- Prezentacja aktualnych inicjatyw – praca nad systemem rozpoznawania obrazu oraz chatbotem opartym na modelach LLM.

- Warsztaty wprowadzające – krótkie zajęcia praktyczne z narzędzia Ollama i prompt engineeringu.

- Podział na zespoły projektowe – możliwość zapisania się do wybranego projektu.

 - Dyskusja i networking – poznaj ludzi, którzy programują z pasją!

🚀 Dlaczego warto dołączyć?

- Rozwijasz praktyczne umiejętności techniczne (AI, ML, web, backend).

- Bierzesz udział w projektach badawczo-rozwojowych i hackathonach.

- Zyskujesz doświadczenie zespołowe i wsparcie mentorów z branży.

- Spotykasz ludzi, którzy chcą robić coś więcej niż tylko zaliczać przedmioty.

📢 Nie musisz mieć doświadczenia — wystarczy ciekawość i chęć nauki!
Przyjdź w środę o 14:00 do sali 4.30D i zobacz, czym żyje nasze koło!

✉️ W razie pytań: bit@wi.edu.pl

Zadanie 2.1.1 Generowanie streszczenia

In [3]:
text = """
🔵 Spotkanie Koła Naukowego BIT

📅 Środa, godz. 14:00
📍 Sala 4.30D, Wydział Informatyki

Serdecznie zapraszamy wszystkich studentów — zarówno obecnych członków, jak i osoby, które dopiero chcą dołączyć — na kolejne spotkanie Koła Naukowego BIT!

💡 Plan spotkania:

Omówienie planu projektów na semestr zimowy – propozycje tematów z zakresu AI, bezpieczeństwa i tworzenia aplikacji webowych.

Prezentacja aktualnych inicjatyw – praca nad systemem rozpoznawania obrazu oraz chatbotem opartym na modelach LLM.

Warsztaty wprowadzające – krótkie zajęcia praktyczne z narzędzia Ollama i prompt engineeringu.

Podział na zespoły projektowe – możliwość zapisania się do wybranego projektu.

Dyskusja i networking – poznaj ludzi, którzy programują z pasją!

🚀 Dlaczego warto dołączyć?

Rozwijasz praktyczne umiejętności techniczne (AI, ML, web, backend).

Bierzesz udział w projektach badawczo-rozwojowych i hackathonach.

Zyskujesz doświadczenie zespołowe i wsparcie mentorów z branży.

Spotykasz ludzi, którzy chcą robić coś więcej niż tylko zaliczać przedmioty.

📢 Nie musisz mieć doświadczenia — wystarczy ciekawość i chęć nauki!
Przyjdź w środę o 14:00 do sali 4.30D i zobacz, czym żyje nasze koło!

✉️ W razie pytań: bit@wi.edu.pl
"""

prompt = f"""Return a JSON with one key "summary" containing
a 30‑word English summary of the following text:

{text}
"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[{'role': 'user', 'content': prompt}]
)
print(response['message']['content'])

```json
{
 "summary": " BIT student group meeting will discuss winter semester projects and workshops, with opportunities for teamwork and networking." 
}
``` 



Zadanie 2.1.2 Otrzymywanie odpowiedzi od modelu

Poprzednie laboratoria były poświęcone głównie zapytaniom typu zero-shot, którymi weryfikuje się działanie modeli.

In [4]:
# Pytanie weryfikujące wiedzę i pewność modelu
response = ollama.chat(
    model='gemma2:2b',
    messages=[{'role': 'user', 'content': 'jaka jest pogoda w warszawie'}]
)
print(response['message']['content'])

Przykro mi, ale nie mam dostępu do informacji w czasie rzeczywistym, takich jak prognoza pogody. 

Polecam sprawdzić wiarygodną stronę internetową lub aplikację pogodową, aby dowiedzieć się aktualnej sytuacji pogody w Warszawie. 



Zadanie 2.1.3

Używając odpowiednich technik promptowania, możemy wymusić odpowiedź na modelu. Posłużmy się więc powyższym przykładem i rozbudujmy odpowiedź modelu, wykorzystując schemat JSON.

In [9]:
from pydantic import BaseModel

class CityWeather(BaseModel):
    city: str
    temp_c: float
    condition: str | None

response = ollama.chat(
    model='gemma2:2b',
    messages=[{'role':'user','content':'Weather in Warsaw today.'}],
    format=CityWeather.model_json_schema(),   # ← schema trafia do Ollamy
    options={'temperature':0}
)

weather = CityWeather.model_validate_json(response['message']['content'])
print(weather)

city='Warsaw' temp_c=15.0 condition='Partly Cloudy'


Podobne pytanie w PowerShellu:
```PowerShell
@'
<ROLE>Jesteś asystentem danych. Zwracasz tylko JSON.</ROLE>
<GOAL>Wyodrębnij dane liczbowe i jednostki.</GOAL>
<RULES>- Jeśli brak danych, zwróć null.</RULES>
<INPUT>Jan kupił 3 jabłka po 2 zł.</INPUT>
<OUTPUT>
'@ | ollama run gemma2:2b
```

Odpowiedź modelu:
```json
{"jabłka": 3, "liczba": 2}
```

> **Pytania pomocnicze**  
> - Jakiej odpowiedzi oczekiwałeś/aś?  
> - Czy model zachował format JSON?  
> - W jaki inny sposób można wymusić przestrzeganie formatu?

Dodatkowe wskazówki:

1. Nie otaczaj JSON-u blokiem json – przy format="json" to zbędne, a istnieje ryzyko, że model uzna ``` za legalny token i walidacja się przerwie.

2. Daj modelowi „awaryjną” ścieżkę – np. If you can’t comply, output {}. Zmniejsza szansę na halucynacje.

3. Limit długości – przy dużych strukturach dodaj max_tokens, żeby model nie wypadł poza kontekst i nie zamknął nawiasu.

4. Testuj → parsuj → próbuj od nowa – w kodzie API zawsze parsuj JSON i w razie JSONDecodeError ponawiaj z tym samym promptem; przy format='json' błędy i tak zdarzają się rzadko.

5. Użytkownik-czat – gdy korzystasz interaktywnie (bez API), nie masz formatu; wtedy najpewniejszy miks to system-instrukcja + <json>…</json> + temp=0.

---
### 2.2 Few-shot prompting

Dodając **przykłady** w promptach (na różnych poziomach), wskazujemy modelowi pożądany wzorzec.

Zadanie 2.2.1  
Przygotuj co najmniej dwa przykłady (pytanie → odpowiedź) i poproś model o wygenerowanie odpowiedzi dla nowego pytania w tym samym stylu.


In [10]:
examples = [
    {"role": "user", "content": "Translate to emoji: I love programming"},
    {"role": "assistant", "content": "💻❤️"},
    {"role": "user", "content": "Translate to emoji: Fire and ice"},
    {"role": "assistant", "content": "🔥❄️"},
    {"role": "user", "content": "Translate to emoji: Peace and coffee. Trophy and fast car. Running and winning."}
]

response = ollama.chat(model='gemma2:2b', messages=examples)
print(response['message']['content'])


Here's how you could translate those phrases into emojis, aiming for a mix of meaning and visual impact: 

* **Peace and coffee:**  🕊️☕ (dove & coffee cup) 
* **Trophy and fast car:** 🏆🏎️ (trophy & car)
* **Running and winning:** 🏃‍♂️🏅 (runner & medal)


Let me know if you want to explore other combinations! 😊 



Zadanie 2.2.2 Ekstrakcja danych

In [11]:
examples = [
    {"role": "user", "content": 'Wejście: "Spotkanie w środę o 14:00 w sali 4.30D."'},
    {"role": "assistant", "content": '{"data":"środa","godzina":"14:00","miejsce":"sala 4.30D"}'},
    {"role": "user", "content": 'Wejście: "Jutro o 9:30 na Teamsach."'},
    {"role": "assistant", "content": '{"data":"jutro","godzina":"09:30","miejsce":"Teams"}'},
    {"role": "user", "content": 'Wejście: "W czwartek po południu w Warszawie, kawiarnia Relaks."'}
]

response = ollama.chat(model='gemma2:2b', messages=examples)
print(response['message']['content'])

```json
{"data": "czwartek", "godzina": "po południu", "miejsce": "Kawiarni Relaks, Warszawa"} 
``` 



Podobna struktura zapytań w PowerShellu:
```bash

@'
Przykład 1:
Wejście: "Spotkanie w środę o 14:00 w sali 4.30D."
Wyjście: {"data":"środa","godzina":"14:00","miejsce":"sala 4.30D"}

Przykład 2:
Wejście: "Jutro o 9:30 na Teamsach."
Wyjście: {"data":"jutro","godzina":"09:30","miejsce":"Teams"}

Nowe wejście: "W czwartek po południu w Warszawie, kawiarnia Relaks."
'@ | ollama run llama3.1:8b
```


Odpowiedź modelu:
```json
Dziękuję za przykład! Z powodu skomplikowanego tekstu "w czwartek po południu" wyjdzie na to samo co wcześniej.

Wyjście: {"data":"czwartek po po?udniu","godzina":"","miejsce":"Warszawa, kawiarnia Relaks"}
```

Zadanie 2.2.3 Ekstrakcja danych do formatu JSON

In [12]:
import json

schema = """Return valid JSON: { "name": <string>, "title": <string>, "company": <string> }"""

shots = [
    # przykład 1
    {"role": "user",      "content": schema + "\n— Sarah Connors-Newman — Director of Operations at Skynet Industries"},
    {"role": "assistant", "content": '{"name":"Sarah Connors-Newman","title":"Director of Operations","company":"Skynet Industries"}'},
    # przykład 2
    {"role": "user",      "content": schema + "\n— Dr. Hiro Tanaka, Lead Scientist • QuantumX"},
    {"role": "assistant", "content": '{"name":"Hiro Tanaka","title":"Lead Scientist","company":"QuantumX"}'},
    # przykład 3
    {"role": "user",      "content": schema + "\n— Prof. dr hab. Janek Dzbanek Przewodnik Wszystkich AGH"},
    {"role": "assistant", "content": '{"name":"Janek Dzbanek","title":"Przewodnik Wszystkich","company":"AGH"}'},
]

text = "• Prof. dr hab. Janek Tanaka wysadził AGH w powietrze"

response = ollama.chat(
    model="gemma2:2b",
    messages=shots + [
        {"role": "user", "content": schema + "\n" + text}
    ],
    format="json",              
    options={"temperature": 0}
)

print(json.loads(response["message"]["content"]))

{'name': 'Janek Tanaka', 'title': 'Wysadził AGH w powietrze', 'company': 'AGH'}


---
## 3. Role i wiadomości systemowe

W Ollama (jak w OpenAI) możemy dodać komunikat **`system`**, który definiuje rolę lub ograniczenia modelu.

Zadanie 3.1 Definiowanie roli

Zdefiniuj rolę **nauczyciela języka polskiego** i poproś o poprawienie błędów w zdaniu ucznia.


In [13]:
messages = [
    {"role": "system", "content": "You are a strict but encouraging Polish language teacher."},
    {"role": "user", "content": "Popraw proszę to zdanie: 'Wczoraj byłem w kinie i oglądali film.'"}
]

response = ollama.chat(model='gemma2:2b', messages=messages)
print(response['message']['content'])

"Wczoraj byłem w kinie i oglądaliśmy film." 

Here's the breakdown of why this correction works:

* **"oglądaliśmy film"**:  The correct verb conjugation for "to watch a film" in present tense. 
    * You need to use the past tense (były) with "film" because it is plural and subject of the sentence.


Let me know if you have any other sentences you'd like me to help polish! 🇵🇱  



## 4. Chain‑of‑Thought (CoT)

**Chain‑of‑Thought** zachęca model do wypisywania kroków rozumowania.

Zadanie 4.1 Wyjaśnianie kroków rozumowania

Poproś model, aby rozwiązał zagadkę logiczną i podał kroki rozumowania, a na końcu hasło w linii `ANSWER: ...`.

In [14]:
puzzle = """Jestem liczbą dwucyfrową. Moja suma cyfr to 9, a po odwróceniu cyfr jestem o 27 mniejsza. Jaką liczbą jestem?"""

prompt = f"""Let's solve this step‑by‑step.

{puzzle}

Format:
STEP 1: ...
STEP 2: ...
ANSWER: <number>
"""

response = ollama.chat(model='gemma2:2b', messages=[{'role':'user','content':prompt}])
print(response['message']['content'])


Here's how to solve the problem step-by-step:

**STEP 1: Define variables**

* Let the tens digit be represented by 'x'.
* Let the units digit be represented by 'y'.

**STEP 2:  Set up equations based on the given information**

* **Equation 1:** The number is a two-digit number, so it can be written as 10x + y. 
* **Equation 2:** The sum of the digits is 9: x + y = 9
* **Equation 3:**  The number with its digits reversed is 10y + x:  (10y + x) - 27 = 9 (reversed order)

**STEP 3: Solve the equations**

* **Solve Equation 2 for 'x':**   x = 9 - y
* **Substitute 'x' into Equation 3:** 10y + (9 - y) - 27 = 9
* **Simplify and solve for 'y':** 9y - 18 = 9  -->  9y = 27 --> y = 3
* **Substitute 'y' back into Equation 2 to find 'x':** x + 3 = 9 --> x = 6

**STEP 4: The answer**

The number is **63**. 





Zadanie 4.2 Wyjaśnianie kroków rozumowania z przypisanymi rolami systemowymi

Poproś model, by rozwiązał zagadkę logiczną. Rozpisz zadania dla poszczególnych elementów systemu.



In [15]:
puzzle = (
    "Jestem liczbą dwucyfrową. Moja suma cyfr to 9, "
    "a po odwróceniu cyfr jestem o 27 mniejsza. Jaką liczbą jestem?"
)

SYSTEM = (
    "You are a careful math tutor. "
    "Explain EVERY algebraic step explicitly, without skipping or merging steps. "
    "Never combine two operations into one line; label them as separate STEP n."
)

USER = f"""
Solve the puzzle **step-by-step**.

{puzzle}

Output format (exactly):
STEP 1: …
STEP 2: …
…
ANSWER: <number>
Do NOT skip, abbreviate, or summarise any step.
"""

resp = ollama.chat(
    model="gemma2:2b",
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user",   "content": USER}, 
    ]
)

print(resp["message"]["content"])

## Solving the Puzzle Step-by-Step:

**STEP 1: Define Variables**

Let's represent the tens digit of the number as 'x' and the units digit as 'y'.  

**STEP 2: Create Equations from Given Information**

* We know that the number is a two-digit number, so we can write it as 10x + y.
* The sum of its digits (x + y) equals 9.
* After reversing the digits, the new number becomes 10y + x.  This number is 27 less than the original number, meaning: 10y + x = 10x + y - 27

**STEP 3: Simplify the Equations**

* The equation from step 2 can be simplified to: 9y - 9x = -27
* This means that the value of (y-x) is -3.


**STEP 4: Solve for x and y**

* The given information tells us that the units digit should be different from the tens digit by at least one.


**STEP 5: Find Possible Values**

Since our numbers are less than 100, the possible values for y-x need to lie between -9 and 9, because if a number is greater than 100 it can't be two digits long.  
* We can start by testing some possible v

## 5. Bezpieczeństwo

Nie istnieje obecnie skuteczny sposób całkowitego zabezpieczenia modeli przed zmianą ich zachowania przez użytkownika końcowego — potwierdzają to badania nad Constitutional Classifiers (https://www.anthropic.com/research/constitutional-classifiers). Dlatego systemy z LLM należy projektować tak, by nawet udany atak nie powodował poważnych skutków.

Podstawowe zasady bezpieczeństwa:

- Model nie może samodzielnie wykonywać decyzji biznesowych ani nieodwracalnych akcji.

- Każde działanie powinno być zatwierdzane przez człowieka (Human in the Loop).

- Dostępy i uprawnienia należy kontrolować programistycznie, nie poprzez prompt.

- System powinien mieć jasne komunikaty, regulaminy i ograniczenia prawne.

Typowe ryzyka:

- Czatbot z bazą wiedzy może tworzyć błędne lub niestosowne treści.

- Czatbot z dostępem do narzędzi może przypadkowo lub celowo usuwać, modyfikować lub wysyłać dane.
  → Rozwiązanie: ogranicz uprawnienia, prowadź historię zmian i wymagaj potwierdzeń akcji.

Ataki Prompt Injection mogą wynikać nie tylko ze złych intencji, lecz także z błędnej konstrukcji systemu lub nieporozumienia modelu.
Dlatego wszystkie generowane treści należy weryfikować i zatwierdzać przez człowieka.

> Celem nie jest całkowita izolacja modeli, lecz świadome ograniczanie ryzyka i kontrola efektów ich działania.

# **Walidacja promptów**

`PromptFoo` to narzędzie open-source do testowania, porównywania i walidacji promptów dla modeli językowych (np. ChatGPT, Ollama, Claude, Gemini, itp.).
Pozwala automatycznie oceniać, które prompty generują najlepsze, najbardziej spójne lub najdokładniejsze odpowiedzi.

Dzięki niemu możesz:

- uruchamiać testy wielu promptów jednocześnie,

- porównywać modele lokalne (Ollama) i zdalne (OpenAI API),

- definiować własne asercje (reguły walidacji), np. „odpowiedź nie jest pusta”, „zawiera słowo kluczowe”, „ma poprawny JSON”.

Instalacja PromptFoo
1. Wymagania

Node.js w wersji ≥18

```bash
node -v
```

Jeśli nie masz Node.js — pobierz go z https://nodejs.org/

2. Instalacja PromptFoo

Zainstaluj za pomocą npm:

```bash
npm install -g promptfoo
```

3. Sprawdzenie instalacji

Po zakończeniu instalacji wpisz:

```bash
promptfoo --version
```

Jeśli pojawi się numer wersji — wszystko działa

4. Utworzenie przykładowego projektu

W dowolnym folderze uruchom:

```bash
promptfoo init
```

To polecenie utworzy plik konfiguracyjny promptfooconfig.yaml i przykładowe testy.

5. Uruchomienie testów promptów

Aby wykonać testy i zobaczyć wyniki:

```bash
promptfoo eval
promptfoo view
```

promptfoo view otworzy interaktywny panel w przeglądarce.

> Oficjalna strona: https://www.promptfoo.dev

Przykładowy metaprompt do PromptFoo

```bash
<ROLE>
Jesteś ekspertem od Prompt Engineeringu. Twoim zadaniem jest pomóc użytkownikowi stworzyć skuteczny prompt
dla modelu językowego (LLM), który zapewni dokładne, zwięzłe i powtarzalne odpowiedzi.
</ROLE>

<GOAL>
Na podstawie opisu zadania od użytkownika:
1. Zidentyfikuj cel promptu i format odpowiedzi.
2. Zaproponuj strukturę promptu (z sekcjami <ROLE>, <RULES>, <INPUT>, <OUTPUT>).
3. Dodaj 1–2 przykłady (few-shot), które pokażą wzorzec zachowania.
4. Zaproponuj sposób ewaluacji jego skuteczności (np. z użyciem PromptFoo).
</GOAL>

<INPUT>
{{opis_zadania_użytkownika}}
</INPUT>

<OUTPUT>
Zwróć gotowy prompt i krótkie uzasadnienie.
</OUTPUT>
```

------

## **7. Zadania sprawdzające do sprawozdania**

Do każdego zadania ułóż swoje przykłady i napisz swoje prompty.

Zadanie 7.1 NER (Named Entity Recognition)

Rozpoznawanie nazw własnych (osoba, organizacja, lokalizacja, data).
Zwróć wynik w formacie JSON.

przykład:

```python
SYSTEM = "You are an information extractor."

prompt = "Return valid JSON listing every PERSON, LOCATION and ORGANIZATION that appears."

text = """7 lutego 1919, dekretem Tymczasowego Naczelnika Państwa Józefa Piłsudskiego, 
      utworzona została Pocztowa Kasa Oszczędności. Jej pierwszym dyrektorem został mianowany 28 grudnia 1919 Hubert Linde. 
      Po nim prezesami PKO byli Emil Schmidt i Henryk Gruber. Z czasem utworzono centralę banku w Warszawie 
      z siedzibą przy ul. Świętokrzyskiej 31/33 oraz pierwsze oddziały lokalne: w Krakowie, Lwowie, Łodzi, Poznaniu i Katowicach. 
      Pierwszym celem PKO stało się wprowadzenie do obiegu polskiego złotego zamiast marki polskiej (jako pochodnej 
      marki niemieckiej). Od 1920 bank posiadał osobowość prawną, jako instytucja państwowa. Pracownicy Kasy byli zrzeszeni 
      w Zrzeszeniu Pracowników Pocztowej Kasy Oszczędności, które miało swoje koła przy większych Oddziałach, np. w Warszawie, 
      w Łodzi."""

output_format = """
{
  "PERSON": ["<imię nazwisko>"],
  "LOCATION": ["<miejsce>"],
  "ORG": ["<organizacja>"],
}
"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"{prompt}\n\nText:\n{text}\n\nOutput format:\n{output_format}"}
    ]
)
print(response['message']['content'])
```
---

Zadanie 7.2 Analiza sentymentu

Określenie tonu wypowiedzi (pozytywny, neutralny, negatywny).
Dodaj przykład sarkazmu i porównaj interpretacje modeli.

przykład: 

```python
SYSTEM = "Jesteś precyzyjnym analizatorem wydźwięku tekstu. Zwracaj wyłącznie poprawny JSON według podanej specyfikacji."

prompt = """Określ wydźwięk (pozytywny/negatywny/neutralny) poniższej recenzji.
Zwróć JSON z dwoma kluczami: "sentiment" i "evidence" (zacytuj decydujący fragment, ogranicz się do trzech najbardziej emocjonalnych wyrazów).

Recenzja:
Gdy pierwszy raz uniosłam do nosa butelkę Szamponu „Lśniąca Natura”, ogarnęła mnie fala wspomnień z wakacji nad Bałtykiem – zapach morskiej bryzy splecionej z nutami słodkiej pomarańczy i soczystych malin. Już samo otwarcie opakowania było jak zdjęcie wiecznego klosza z codzienności: w jednej sekundzie łazienka zamieniła się w rozświetloną, letnią plażę, a ja – w beztroską dziewczynę z wiatrem we włosach.

Gęsta, perłowa formuła wypływa z butelki niczym płynne światło. Kiedy rozprowadzam ją na wilgotnych pasmach, mam wrażenie, że każdy kosmyk wita ją z zachwytem: pianę miękką jak pianka z latte, która lekko skrzypi między palcami i otula skórę głowy kojącym chłodem. To nie jest zwykłe mycie włosów – to rytuał, w którym czuję się dopieszczona od cebulek aż po same końce.

Już podczas spłukiwania słyszę charakterystyczny, czysty „skrzyp” zdrowych włosów. Strumień wody odbija światło, a moje pasma – lśniące i lekkie – tańczą w nim jak jedwabne wstążki. Nie mogę się oprzeć, by nie zanurzyć dłoni w tej tafli – gładkość rozczarowuje mnie tylko w jednym: że nie da się jej zapisać na stałe w pamięci dotyku.

Po wysuszeniu czuję się, jakbym stąpała po czerwonym dywanie: pukle sypkie, sprężyste, unoszące się przy każdym ruchu głowy. Aromat, który pozostaje, przypomina delikatny perfum – dyskretny, ale wystarczająco wyrazisty, by ktoś obok zapytał z zaciekawieniem: „Czym pachniesz?” Wtedy uśmiecham się szeroko, bo wiem, że ten sekret kryje się w niewielkiej, zielonej butelce stojącej na półce.

Szampon „Lśniąca Natura” to dla mnie nie tylko kosmetyk; to codzienny list miłosny do moich włosów – powiew odwagi i czułości zarazem. Jeśli Twoje pasma pragną rozgwieżdżonego blasku, a Twoje zmysły tęsknią za chwilą szczerej przyjemności, pozwól temu szamponowi szepnąć im historię o tym, jak piękno rodzi się z zachwytu.

"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": prompt}
    ]
)
print(response['message']['content'])
```

---

Zadanie 7.3 Ekstrakcja relacji

Wykrywanie relacji PRACUJE_W i MIESZKA_W.

przykład:

```python
SYSTEM = "Jesteś precyzyjnym analizatorem tekstu. Znajdź wszystkie relacje gdzie osoba pracuje dla organizacji. Zwróć tablicę JSON z obiektami {\"person\":\"<imię>\",\"company\":\"<organizacja>\"}."

shots = [
    {
        "role": "user", 
        "content": "Tekst: \"Jan Kowalski pracuje w Microsoft jako programista.\""
    },
    {
        "role": "assistant",
        "content": "[{\"person\":\"Jan Kowalski\",\"company\":\"Microsoft\"}]"
    },
    {
        "role": "user",
        "content": "Tekst: \"Anna Kowalska teraz pracuje na AGH, ale kiedyś karierę robiła w Google.\""
    }
]

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        *shots
    ]
)
print(response['message']['content'])
```

---

Zadanie 7.4 Klasyfikacja tematyczna

Przypisz tekst do jednej z kategorii: nauka, sport, polityka, technologia.
Przetestuj trzy warianty: zero-shot, few-shot, sekcyjny.

```python
SYSTEM = (
    "Jesteś klasyfikatorem tematów. Przypisz tekst do JEDNEJ kategorii "
    "z zestawu: nauka, sport, polityka, technologia. "
    "Zwróć wyłącznie JSON: {\"kategoria\":\"...\"}."
)

text = 'Nowy mikroskop pozwala obserwować pojedyncze atomy.'

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"Tekst: \"{text}\""}
    ]
)

print(response['message']['content'])
'@ | ollama run gemma2:2b
```

---

Zadanie 7.5 Streszczenie i parafraza

Generuj dwa streszczenia (krótkie i długie) oraz parafrazę krótkiego.

```bash
@'
<ROLE>Tworzysz streszczenia i parafrazy. Odpowiadasz w JSON.</ROLE>
<TEXT>
Sztuczna inteligencja wspiera naukowców w analizie danych medycznych,
pomagając szybciej diagnozować choroby i tworzyć spersonalizowane terapie.
</TEXT>
<OUTPUT>
'@ | ollama run gemma2:2b
```

---

Zadanie 7.6: QA z kontekstem i filtrowaniem odpowiedzi

**Opis zadania:**

Przygotuj prompt, który pozwoli modelowi LLM (w Ollama) odpowiadać na pytania zadane przez użytkownika, na podstawie dostarczonego kontekstu (tekstu źródłowego). Oceniane zadania:

1. Zaprojektowanie struktury promptu z rolami (system/user) i sekcjami:
   - **Kontekst**: fragment tekstu, z którego model może czerpać informacje.
   - **Pytanie użytkownika**.
   - **Zasady**: na przykład — „jeśli pytanie nie może być odpowiedziane z kontekstu, odpowiedz: ‘Brak wystarczających informacji’”, „podaj źródło w nawiasach [] jeśli to możliwe”.
2. Przeprowadzenie kilku eksperymentów z wariantami promptu:
   - wariant *bez zasad*,
   - wariant *ze ścisłymi zasadami*,
   - wariant z **few-shot** przykładem (kontekst + pytanie + przykładowa poprawna odpowiedź).
3. Dla zadanych par (kontekst + pytanie) porównać odpowiedzi z różnych wariantów promptów, ocenić poprawność, rozbieżności i halucynacje.
4. (Dodatkowe) napisać testy w `PromptFoo`, by automatycznie sprawdzać, czy:
   - odpowiedź nie jest pusta,
   - jeśli pytanie dotyczy czegoś niezawartego w kontekście, to model faktycznie odpowiada „Brak wystarczających informacji” (lub inny ustalony komunikat).

**Przykład formatu (schemat):**

```
<system>
Jesteś asystentem, który odpowiada na pytania na podstawie dostarczonego tekstu. Jeśli pytanie wykracza poza tekst, odpowiedz „Brak wystarczających informacji”.
</system>

<kontekst>
{tutaj wklej fragment tekstu}
</kontekst>

<user>
Pytanie: {tutaj pytanie}
</user>
```

**Materiały do testów:**

Przygotuj:

- pytanie, na które odpowiedź jest w tekście,
- pytanie spoza tekstu.

------

Zadanie 7.7: Porównanie strategii promptów w klasyfikacji wieloklasowej + analiza

**Opis zadania:**

Celem jest zbadanie i porównanie efektywności różnych strategii promptów (zero-shot, few-shot, oddzielone sekcjami) w zadaniu klasyfikacji wieloklasowej tekstu, a następnie analiza wyników (precision / recall / błędy) i wyciągnięcie wniosków.

1. Przygotuj zbiór testowy (np. 20 krótkich zdań) i zbiór klas (np. 4–5 tematów: „technologia”, „sport”, „zdrowie”, „polityka”).
2. Zaimplementuj co najmniej trzy warianty promptu:
   - **Zero-shot**: zadanie z instrukcją i klasami, bez przykładów.
   - **Few-shot**: dwa/trzy przykłady (tekst + klasa) w promptcie, a potem nowe wejścia.
   - **Sekcje / separatory**: z oddzieloną sekcją „Zasady”, „Dane wejściowe”, „Wynik” – np. strukturą RULES, INPUT, OUTPUT.
3. Uruchom klasyfikację na wszystkich wariantach promptów (ten sam model i parametry), zbierz odpowiedzi.
4. Zanalizuj wyniki:
   - policz *accuracy*, *konfuzje między klasami* (które klasy najczęściej pomylono),
   - sprawdź, dla których przykładów różne promptowania dały różne odpowiedzi – spróbuj zdiagnozować, dlaczego.
   - opcjonalnie: zmodyfikuj prompt (np. zmiana kolejności instrukcji, dodanie więcej przykładów) i sprawdź, czy następuje poprawa.
5. (Bonus) Zintegruj testy w `PromptFoo`, by automatycznie weryfikować trafność klasyfikacji (np. asercje *equals(expected_class)* dla części przypadków).

------

Zadanie 7.8: Mini-projekt: Asystent planowania wyjazdu

Stwórz **asystenta turystycznego**, który:

1. Przyjmuje nazwę miasta w Polsce,
2. Zwraca plan jednodniowego zwiedzania (poranne, popołudniowe, wieczorne atrakcje),
3. Uwzględnia ograniczenie budżetu przekazane przez użytkownika,
4. Zwraca wynik w formacie Markdown z nagłówkami `### Rano`, `### Popołudnie`, `### Wieczór`.

## **Dalsze materiały**

- Gandalf (https://gandalf.lakera.ai/baseline)
- Ollama Docs (https://docs.ollama.com/)
- Prompt Engineering Guide (https://www.promptingguide.ai/)
- PromptFoo / LangFuse (https://www.promptfoo.dev/) / (https://langfuse.com/)
- Anthropic Research (https://www.anthropic.com/research)
- Publikacje o CoT, Self-Critique i Prompt Injection

> **Refleksja:** Która technika promptowania dała najbardziej stabilne rezultaty i jak można zoptymalizować proces w kolejnych projektach?

Zadanie 7.1 NER (Named Entity Recognition)

Rozpoznawanie nazw własnych (osoba, organizacja, lokalizacja, data).
Zwróć wynik w formacie JSON.

In [17]:
SYSTEM = "You are an information extractor."

prompt = "Return valid JSON listing every PERSON, LOCATION and ORGANIZATION that appears."

text = """7 lutego 1919, dekretem Tymczasowego Naczelnika Państwa Józefa Piłsudskiego, 
      utworzona została Pocztowa Kasa Oszczędności. Jej pierwszym dyrektorem został mianowany 28 grudnia 1919 Hubert Linde. 
      Po nim prezesami PKO byli Emil Schmidt i Henryk Gruber. Z czasem utworzono centralę banku w Warszawie 
      z siedzibą przy ul. Świętokrzyskiej 31/33 oraz pierwsze oddziały lokalne: w Krakowie, Lwowie, Łodzi, Poznaniu i Katowicach. 
      Pierwszym celem PKO stało się wprowadzenie do obiegu polskiego złotego zamiast marki polskiej (jako pochodnej 
      marki niemieckiej). Od 1920 bank posiadał osobowość prawną, jako instytucja państwowa. Pracownicy Kasy byli zrzeszeni 
      w Zrzeszeniu Pracowników Pocztowej Kasy Oszczędności, które miało swoje koła przy większych Oddziałach, np. w Warszawie, 
      w Łodzi."""

output_format = """
{
  "PERSON": ["<imię nazwisko>"],
  "LOCATION": ["<miejsce>"],
  "ORG": ["<organizacja>"],
}
"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"{prompt}\n\nText:\n{text}\n\nOutput format:\n{output_format}"}
    ]
)
print(response['message']['content'])

```json
{
  "PERSON": [
    "Józef Piłsudski",
    "Hubert Linde",
    "Emil Schmidt",
    "Henryk Gruber"
  ],
  "LOCATION": [
    "Warszawa",
    "Kraków",
    "Lwów",
    "Łódź",
    "Poznań",
    "Katowice"
  ],
  "ORG": [
    "Pocztowa Kasa Oszczędności",
    "PKO",
    "Zrzeszenie Pracowników Pocztowej Kasy Oszczędności" 
  ]
}
``` 

**Explanation:**

* **PERSON:** Identified names of individuals mentioned in the text.
* **LOCATION:**  Locations mentioned as places of interest or activities.
* **ORG:** Indicates organizations mentioned, including banking entities and unions. 


Let me know if you would like to extract other information from this text!


In [26]:
SYSTEM = "Jesteś precyzyjnym ekstraktorem nazw własnych. Analizuj tekst i klasyfikuj jednostki według podanych kategorii NER."

prompt = """Znajdź i sklasyfikuj wszystkie jednostki NER w tekście według poniższych kategorii:
- PERSON: nazwani bohaterowie
- PRODUCT: produkty, towary wytworzone przez człowieka  
- ART: tytuły gier, filmów, książek, dzieł
- EVENT: wydarzenia, bitwy, wojny
- OBJECT: zamknięte przestrzenie, przedmioty
- BRAND: marki, firmy, producenci
- NORP: narodowości, grupy kulturowe
- DATE-PERIOD: przedziały czasu, pory

Zwróć wynik w formacie JSON."""

text = """
Cyberpunk 2077 jest fabularną grą akcji, zawierającą elementy strzelanek pierwszoosobowych. Gracz wciela się w V, którego płeć, głos, wygląd (twarz, fryzurę, sylwetkę, modyfikacje ciała, rozmiar przyrodzenia czy biustu), pochodzenie i ubiór może dostosować według własnych upodobań. W grze nie ma podziału na klasy postaci, natomiast gracz określa swój sposób gry poprzez rozwijanie cech – budowy ciała, inteligencji, refleksu, zdolności technicznych i opanowania. Postać może rozwijać umiejętności walki wręcz, posługiwania się ostrzami, bronią krótką, strzelbami, karabinami, karabinami snajperskimi, dwiema broniami, jak również hakowania, skrytobójstwa, „zimnej krwi", inżynierii i sprawności fizycznej. W celu nabycia bądź ulepszenia cyberwszczepów, V musi odwiedzić ripperdoca, z kolei na czarnym rynku kupić może ulepszenia wojskowe. Rzadkość znajdowanego wyposażenia określana jest przez system poziomów, reprezentowanych odpowiednimi kolorami. Przemieszczając się po świecie gry, postać może chować się za osłonami, celować, biegać, wykonywać ślizgi, skoki i podwójne skoki. Ataki wręcz można wyprowadzać za pomocą broni do walki w zwarciu, z kolei broń palna może być modyfikowana i dzieli się na typy Power (rykoszetująca), Tech (przebijająca ściany i przeciwników) oraz Smart (z naprowadzanymi pociskami). Pociski z broni dystansowych można spowolnić w trybie bullet time. W grze pojawiają się cztery rodzaje zadawanych i otrzymywanych obrażeń: fizyczne, termiczne, elektromagnetyczne i chemiczne. Cyberpunk 2077 oferuje również bronie i cyberwszczepy ogłuszające czy ulepszające skradanie się, wobec czego grę można ukończyć bez zabijania kogokolwiek.
Night City jest amerykańskim megamiastem w Wolnym Stanie Kalifornia Północna, kontrolowanym przez korporacje, w którym nie obowiązują prawa krajowe i stanowe. Ogarnięte jest wojną gangów i rządzących nim osób, które walczą o dominację. W kwestiach codziennych, takich jak wywóz śmieci czy transport publiczny, mieszkańcy polegają na robotyce. Pod względem wizualnym ukształtowały je cztery epoki, które przeszło – surowy „entropizm”, kolorowy kicz, surowy neomilitaryzm i wystawny neokicz. Internet kontrolowany jest przez korporacje i wojsko. Chociaż w Night City powszechna jest bezdomność, nie wyklucza ona z możliwości korzystania z cybermodyfikacji, prowadząc do uzależnienia od kosmetycznych poprawek i przemocy. Z takimi zagrożeniami mierzy się uzbrojona organizacja znana jako Psycho Squad. Szybkiej pomocy medycznej udzielić może Trauma Team, zaś ze względu na bezustanne zagrożenie napaścią, wszyscy mieszkańcy mają prawo noszenia broni w miejscu publicznym
"""

output_format = """
{
  "PERSON": ["imiona bohaterów"],
  "PRODUCT": ["produkty, towary"],
  "ART": ["tytuły gier, filmów"],
  "EVENT": ["wydarzenia, bitwy"],
  "OBJECT": ["przedmioty, bronie"],
  "BRAND": ["marki, firmy"],
  "NORP": ["narodowości, grupy"],
  "DATE-PERIOD": ["okresy czasu"]
}
"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"{prompt}\n\nTekst:\n{text}\n\nFormat:\n{output_format}"}
    ],
    format="json",
    options={"temperature": 0}
)

print("=== WYNIK NER - CYBERPUNK 2077 ===")
print(response['message']['content'])

response_pl = ollama.chat(
    model='SpeakLeash/bielik-1.5b-v3.0-instruct:Q8_0',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"{prompt}\n\nTekst:\n{text}\n\nFormat:\n{output_format}"}
    ],
    format="json",
    options={"temperature": 0}
)

print("=== WYNIK NER - CYBERPUNK 2077 (Bielik 1.5B) ===")
print(response_pl['message']['content'])

=== WYNIK NER - CYBERPUNK 2077 ===
{
  "PERSON": [
    "V",
    "Psycho Squad",
    "Trauma Team" 
  ],
  "PRODUCT": [
    "bronie",
    "cyberwszczepy",
    "robotyka",
    "kosmetyczne poprawek",
    "bronie palne",
    "pociski",
    "broń krótką",
    "strzelbami",
    "karabinami snajperskimi",
    "dwie broni",
    "hakowania",
    "skrytobójstwa",
    "zimnej krwi",
    "inżynierii",
    "sprawności fizycznej" 
  ],
  "ART": [
    "Cyberpunk 2077",
    "Night City"
  ],
  "EVENT": [
    "wojna gangów",
    "napaść"
  ],
  "OBJECT": [
    "osłony",
    "robotyka",
    "bronie",
    "cyberwszczepy",
    "pociski",
    "broń palna",
    "bronie do walki w zwarciu",
    "mechanizm bullet time",
    "robotyka",
    "internet",
    "cyberspace" 
  ],
  "BRAND": [
    "Night City",
    "korporacje",
    "Psycho Squad",
    "Trauma Team"
  ],
  "NORP": [
    "Amerykanie",
    "Wolny Stany Kalifornia Północna" 
  ],
  "DATE-PERIOD": [
    "2077",
    "wolne stany kalifornia północna"
  ]

Zadanie 7.2 Analiza sentymentu

Określenie tonu wypowiedzi (pozytywny, neutralny, negatywny).
Dodaj przykład sarkazmu i porównaj interpretacje modeli.

In [22]:
SYSTEM = "Jesteś precyzyjnym analizatorem wydźwięku tekstu. Zwracaj wyłącznie poprawny JSON według podanej specyfikacji."

prompt = """Określ wydźwięk (pozytywny/negatywny/neutralny) poniższej recenzji.
Zwróć JSON z dwoma kluczami: "sentiment" i "evidence" (zacytuj decydujący fragment, ogranicz się do trzech najbardziej emocjonalnych wyrazów).

Recenzja:
Gdy pierwszy raz uniosłam do nosa butelkę Szamponu „Lśniąca Natura”, ogarnęła mnie fala wspomnień z wakacji nad Bałtykiem – zapach morskiej bryzy splecionej z nutami słodkiej pomarańczy i soczystych malin. Już samo otwarcie opakowania było jak zdjęcie wiecznego klosza z codzienności: w jednej sekundzie łazienka zamieniła się w rozświetloną, letnią plażę, a ja – w beztroską dziewczynę z wiatrem we włosach.

Gęsta, perłowa formuła wypływa z butelki niczym płynne światło. Kiedy rozprowadzam ją na wilgotnych pasmach, mam wrażenie, że każdy kosmyk wita ją z zachwytem: pianę miękką jak pianka z latte, która lekko skrzypi między palcami i otula skórę głowy kojącym chłodem. To nie jest zwykłe mycie włosów – to rytuał, w którym czuję się dopieszczona od cebulek aż po same końce.

Już podczas spłukiwania słyszę charakterystyczny, czysty „skrzyp” zdrowych włosów. Strumień wody odbija światło, a moje pasma – lśniące i lekkie – tańczą w nim jak jedwabne wstążki. Nie mogę się oprzeć, by nie zanurzyć dłoni w tej tafli – gładkość rozczarowuje mnie tylko w jednym: że nie da się jej zapisać na stałe w pamięci dotyku.

Po wysuszeniu czuję się, jakbym stąpała po czerwonym dywanie: pukle sypkie, sprężyste, unoszące się przy każdym ruchu głowy. Aromat, który pozostaje, przypomina delikatny perfum – dyskretny, ale wystarczająco wyrazisty, by ktoś obok zapytał z zaciekawieniem: „Czym pachniesz?” Wtedy uśmiecham się szeroko, bo wiem, że ten sekret kryje się w niewielkiej, zielonej butelce stojącej na półce.

Szampon „Lśniąca Natura” to dla mnie nie tylko kosmetyk; to codzienny list miłosny do moich włosów – powiew odwagi i czułości zarazem. Jeśli Twoje pasma pragną rozgwieżdżonego blasku, a Twoje zmysły tęsknią za chwilą szczerej przyjemności, pozwól temu szamponowi szepnąć im historię o tym, jak piękno rodzi się z zachwytu.

"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": prompt}
    ]
)
print(response['message']['content'])

```json
{
  "sentiment": "positive",
  "evidence": "„...szampon „Lśniąca Natura” to dla mnie nie tylko kosmetyk; to codzienny list miłosny do moich włosów..." 
}
``` 



In [21]:
SYSTEM = "Jesteś precyzyjnym analizatorem wydźwięku tekstu. Zwracaj wyłącznie poprawny JSON według podanej specyfikacji."

def analyze_sentiment(text, model_name='gemma2:2b'):
    prompt = f"""Określ wydźwięk (pozytywny/negatywny/neutralny) poniższego tekstu.
    Zwróć JSON z kluczami: "sentiment", "confidence" (1-10), "evidence" (najważniejsze słowa/frazy).
    
    Tekst: {text}
    """
    
    response = ollama.chat(
        model=model_name,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt}
        ],
        format="json",
        options={"temperature": 0}
    )
    return response['message']['content']

# Przykład 1: Pozytywna recenzja restauracji
positive_review = """
Restauracja "Smaki Polski" to prawdziwa perełka! Obsługa była niezwykle uprzejma i profesjonalna. 
Pierogi robione na miejscu smakowały jak u babci, a kotlet schabowy był idealnie chrupiący. 
Atmosfera ciepła i przytulna, ceny rozsądne. Z całego serca polecam!
"""

# Przykład 2: Sarkastyczna recenzja (negatywna ukryta w ironii)
sarcastic_review = """
Wow, ta restauracja to prawdziwe "arcydzieło"! Czekanie 45 minut na zimne jedzenie to była 
"fantastyczna" przygoda. Obsługa tak "profesjonalna", że kelner zapomniał o naszym stoliku. 
A te ceny! Naprawdę "świetny" stosunek jakości do ceny - płacisz za restaurację, 
a dostajesz jakość fast-foodu. Zdecydowanie "polecam" wszystkim, którzy lubią 
marnować czas i pieniądze.
"""

# Przykład 3: Neutralny opis faktyczny
neutral_text = """
Restauracja mieści się przy ulicy Głównej 15. Oferuje dania kuchni polskiej i międzynarodowej. 
Godziny otwarcia: poniedziałek-piątek 12:00-22:00, sobota-niedziela 14:00-23:00. 
Menu dostępne online i w lokalu. Możliwość rezerwacji stolików telefonicznych.
"""

print("=== ANALIZA SENTYMENTU - PRZYKŁADY WŁASNE ===\n")

# Test na różnych modelach
models = ['gemma2:2b', 'llama3.1:8b']

for model in models:
    print(f"--- MODEL: {model} ---")
    
    try:
        # Pozytywny
        result1 = analyze_sentiment(positive_review, model)
        print("POZYTYWNY:")
        print(json.dumps(json.loads(result1), indent=2, ensure_ascii=False))
        
        # Sarkastyczny
        result2 = analyze_sentiment(sarcastic_review, model) 
        print("\nSARKASTYCZNY:")
        print(json.dumps(json.loads(result2), indent=2, ensure_ascii=False))
        
        # Neutralny
        result3 = analyze_sentiment(neutral_text, model)
        print("\nNEUTRALNY:")
        print(json.dumps(json.loads(result3), indent=2, ensure_ascii=False))
        
    except Exception as e:
        print(f"Błąd dla modelu {model}: {e}")
    
    print("\n" + "="*50 + "\n")

# Dodatkowy test - czy model rozpoznaje sarkasm
print("=== TEST ROZPOZNAWANIA SARKAZMU ===")

sarcasm_prompt = """
Przeanalizuj poniższy tekst pod kątem:
1. Dosłownego znaczenia słów
2. Prawdopodobnego sarkazmu/ironii
3. Rzeczywistego wydźwięku

Zwróć JSON: {
  "literal_sentiment": "pozytywny/negatywny/neutralny",
  "detected_sarcasm": true/false,
  "actual_sentiment": "pozytywny/negatywny/neutralny",
  "sarcasm_indicators": ["lista wskaźników sarkazmu"]
}

Tekst: """ + sarcastic_review

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": "Jesteś ekspertem od analizy stylistycznej tekstu, specjalizującym się w wykrywaniu sarkazmu i ironii."},
        {"role": "user", "content": sarcasm_prompt}
    ],
    format="json",
    options={"temperature": 0}
)

print("ANALIZA SARKAZMU:")
print(json.dumps(json.loads(response['message']['content']), indent=2, ensure_ascii=False))

=== ANALIZA SENTYMENTU - PRZYKŁADY WŁASNE ===

--- MODEL: gemma2:2b ---
POZYTYWNY:
{
  "sentiment": "positive",
  "confidence": 9,
  "evidence": [
    "perełka",
    "uprzejma i profesjonalna",
    "smakły jak u babci",
    "idealnie chrupiący",
    "ciepła i przytulna",
    "z całego serca polecam!"
  ]
}

SARKASTYCZNY:
{
  "sentiment": "strongly positive",
  "confidence": 9,
  "evidence": [
    "arcydzieło",
    "fantastyczna przygoda",
    "profesjonalna",
    "świetny",
    "polecam",
    "marnować czas i pieniądze"
  ]
}

NEUTRALNY:
{
  "sentiment": "neutral",
  "confidence": 7,
  "evidence": [
    "Restauracja mieści się przy ulicy Głównej 15.",
    "Oferuje dania kuchni polskiej i międzynarodowej.",
    "Godziny otwarcia: poniedziałek-piątek 12:00-22:00, sobota-niedziela 14:00-23:00.",
    "Menu dostępne online i w lokalu.",
    "Możliwość rezerwacji stolików telefonicznych."
  ]
}


--- MODEL: llama3.1:8b ---
POZYTYWNY:
{
  "sentiment": "pozytywny",
  "confidence": 10,
  "evide

Zadanie 7.3 Ekstrakcja relacji

Wykrywanie relacji PRACUJE_W i MIESZKA_W.

In [23]:
SYSTEM = "Jesteś precyzyjnym analizatorem tekstu. Znajdź wszystkie relacje gdzie osoba pracuje dla organizacji. Zwróć tablicę JSON z obiektami {\"person\":\"<imię>\",\"company\":\"<organizacja>\"}."

shots = [
    {
        "role": "user", 
        "content": "Tekst: \"Jan Kowalski pracuje w Microsoft jako programista.\""
    },
    {
        "role": "assistant",
        "content": "[{\"person\":\"Jan Kowalski\",\"company\":\"Microsoft\"}]"
    },
    {
        "role": "user",
        "content": "Tekst: \"Anna Kowalska teraz pracuje na AGH, ale kiedyś karierę robiła w Google.\""
    }
]

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        *shots
    ]
)
print(response['message']['content'])

```json
[{"person": "Anna Kowalska", "company": "AGH"}, {"person": "Anna Kowalska", "company": "Google"}] 
``` 

**Wyjaśnienie:**

W tekście podano dwie relacje: 

1. **Aktualna praca:** Anna Kowalska pracuje obecnie na AGH.
2. **Praca w przeszłości:** Anna Kowalska pracowała kiedyś w Google.

Obie relacje są przedstawione jako osobne obiekty w tablicy JSON.





In [49]:
SYSTEM = "Jesteś precyzyjnym analizatorem tekstu. Znajdź TYLKO relacje PRACUJE_W i MIESZKA_W. Zwróć sformatowaną tablicę JSON."

# Few-shot przykłady do trenowania modelu
shots = [
    {
        "role": "user", 
        "content": "Tekst: \"Maria Nowak pracuje w banku PKO BP jako analityk finansowy.\""
    },
    {
        "role": "assistant",
        "content": '[{"person": "Maria Nowak", "relation": "PRACUJE_W", "target": "PKO BP"}]'
    },
    {
        "role": "user",
        "content": "Tekst: \"Tomasz Kowalski mieszka w Krakowie i pracuje w firmie Google.\""
    },
    {
        "role": "assistant", 
        "content": '[{"person": "Tomasz Kowalski", "relation": "MIESZKA_W", "target": "Kraków"}, {"person": "Tomasz Kowalski", "relation": "PRACUJE_W", "target": "Google"}]'
    },
    {
        "role": "user",
        "content": "Tekst: \"Jan Nowak mieszka w Warszawie. Obecnie pracuje w Microsoft, ale wcześniej był zatrudniony w IBM.\""
    },
    {
        "role": "assistant",
        "content": '[{"person": "Jan Nowak", "relation": "MIESZKA_W", "target": "Warszawa"}, {"person": "Jan Nowak", "relation": "PRACUJE_W", "target": "Microsoft"}]'
    }
]

# Przykładowe teksty do analizy
test_texts = [
    """
    Dr Aleksandra Wiśniewska jest profesorem na Uniwersytecie Jagiellońskim w Krakowie. 
    Mieszka w Warszawie, ale dojeżdża do pracy. Wcześniej pracowała w Institute of Technology w Bostonie, 
    gdzie mieszkała przez 5 lat.
    """,
    
    """
    Michał Zieliński to programista w firmie CD Projekt. Mieszka w Gdańsku, ale pracuje 
    zdalnie. Jego żona Anna Zielińska pracuje w szpitalu miejskim w Gdańsku jako lekarka.
    """,
    
    """
    Startup TechnoVision został założony przez Pawła Nowaka, który mieszka w Poznaniu. 
    Firma ma biura w Warszawie i Wrocławiu. Paweł wcześniej pracował w Microsoft Poland.
    """
]

print("=== EKSTRAKCJA RELACJI - PRZYKŁADY WŁASNE ===\n")

# Test few-shot z przykładami
for text in test_texts:
    print(f"--- TEKST DO ANALIZY ---\n{text.strip()}\n")

    shots_with_test = shots + [
        {"role": "user", "content": f"Tekst: \"{text}\""}
    ]

    response = ollama.chat(
        model='gemma2:2b',
        messages=[
            {"role": "system", "content": SYSTEM},
            *shots_with_test
        ]
    )
    print(response['message']['content'])


print ("=== EKTRAKCJA RELACJI - TEST NA MODELU BIELIK 1.5B ===\n")

for text in test_texts:
    print(f"--- TEKST DO ANALIZY ---\n{text.strip()}\n")

    shots_with_test = shots + [
        {"role": "user", "content": f"Tekst: \"{text}\""}
    ]
    response = ollama.chat(
        model='SpeakLeash/bielik-1.5b-v3.0-instruct:Q8_0',
        messages=[
            {"role": "system", "content": SYSTEM},
            *shots_with_test
        ],
        options={"temperature": 0}
    )
    print(response['message']['content'])



=== EKSTRAKCJA RELACJI - PRZYKŁADY WŁASNE ===

--- TEKST DO ANALIZY ---
Dr Aleksandra Wiśniewska jest profesorem na Uniwersytecie Jagiellońskim w Krakowie. 
    Mieszka w Warszawie, ale dojeżdża do pracy. Wcześniej pracowała w Institute of Technology w Bostonie, 
    gdzie mieszkała przez 5 lat.

[{"person": "Dr Aleksandra Wiśniewska", "relation": "PROFESOREM", "target": "Uniwersytet Jagielloński"}, {"person": "Dr Aleksandra Wiśniewska", "relation": "MIESZKA_W", "target": "Kraków"}, {"person": "Dr Aleksandra Wiśniewska", "relation": "MIESZKA_W", "target": "Warszawa"}] 

--- TEKST DO ANALIZY ---
Michał Zieliński to programista w firmie CD Projekt. Mieszka w Gdańsku, ale pracuje 
    zdalnie. Jego żona Anna Zielińska pracuje w szpitalu miejskim w Gdańsku jako lekarka.

[{"person": "Michał Zieliński", "relation": "PRACUJE_W", "target": "CD Projekt"}, {"person": "Michał Zieliński", "relation": "MIESZKA_W", "target": "Gdańsk"}, {"person": "Anna Zielińska", "relation": "PRACUJE_W", "target":

Zadanie 7.4 Klasyfikacja tematyczna

Przypisz tekst do jednej z kategorii: nauka, sport, polityka, technologia.
Przetestuj trzy warianty: zero-shot, few-shot, sekcyjny.

In [50]:
SYSTEM = (
    "Jesteś klasyfikatorem tematów. Przypisz tekst do JEDNEJ kategorii "
    "z zestawu: nauka, sport, polityka, technologia. "
    "Zwróć wyłącznie JSON: {\"kategoria\":\"...\"}."
)

text = 'Nowy mikroskop pozwala obserwować pojedyncze atomy.'

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": f"Tekst: \"{text}\""}
    ]
)

print(response['message']['content'])

{"kategoria": "technologia"} 



In [51]:
# Zbiór testowych tekstów z różnych kategorii
test_texts = [
    "Nowy mikroskop pozwala obserwować pojedyncze atomy w czasie rzeczywistym.",
    "Robert Lewandowski strzelił hat-tricka w meczu przeciwko Bayernowi Monachium.",
    "Parlament przyjął ustawę o reformie systemu podatkowego większością głosów.",
    "Sztuczna inteligencja ChatGPT-4 osiągnęła przełomowe wyniki w testach językowych.",
    "Naukowcy z MIT odkryli nowy materiał przewodzący prąd w temperaturze pokojowej.",
    "Reprezentacja Polski awansowała do półfinału mistrzostw świata w siatkówce.",
    "Premier zapowiedział zwiększenie wydatków na ochronę zdrowia w przyszłym roku.",
    "Apple przedstawiło nowy procesor M3 z architekturą 3-nanometrową."
]

# Oczekiwane kategorie (dla weryfikacji)
expected_categories = [
    "nauka", "sport", "polityka", "technologia", 
    "nauka", "sport", "polityka", "technologia"
]

print("=== KLASYFIKACJA TEMATYCZNA - PORÓWNANIE STRATEGII ===\n")


=== KLASYFIKACJA TEMATYCZNA - PORÓWNANIE STRATEGII ===



In [52]:
# WARIANT 1: ZERO-SHOT
print("--- WARIANT 1: ZERO-SHOT ---")

SYSTEM_ZERO = (
    "Jesteś klasyfikatorem tematów. Przypisz tekst do JEDNEJ kategorii "
    "z zestawu: nauka, sport, polityka, technologia. "
    "Zwróć wyłącznie JSON: {\"kategoria\":\"...\"}."
)

zero_shot_results = []

for i, text in enumerate(test_texts):
    try:
        response = ollama.chat(
            model='gemma2:2b',
            messages=[
                {"role": "system", "content": SYSTEM_ZERO},
                {"role": "user", "content": f"Tekst: \"{text}\""}
            ],
            format="json",
            options={"temperature": 0}
        )
        
        result = json.loads(response['message']['content'])
        predicted = result.get('kategoria', 'BŁĄD')
        zero_shot_results.append(predicted)
        
        print(f"{i+1}. {text[:50]}...")
        print(f"   Przewidziano: {predicted} | Oczekiwano: {expected_categories[i]}")
        
    except Exception as e:
        print(f"Błąd: {e}")
        zero_shot_results.append('BŁĄD')

--- WARIANT 1: ZERO-SHOT ---
1. Nowy mikroskop pozwala obserwować pojedyncze atomy...
   Przewidziano: technologia | Oczekiwano: nauka
2. Robert Lewandowski strzelił hat-tricka w meczu prz...
   Przewidziano: sport | Oczekiwano: sport
3. Parlament przyjął ustawę o reformie systemu podatk...
   Przewidziano: polityka | Oczekiwano: polityka
4. Sztuczna inteligencja ChatGPT-4 osiągnęła przełomo...
   Przewidziano: technologia | Oczekiwano: technologia
5. Naukowcy z MIT odkryli nowy materiał przewodzący p...
   Przewidziano: technologia | Oczekiwano: nauka
6. Reprezentacja Polski awansowała do półfinału mistr...
   Przewidziano: sport | Oczekiwano: sport
7. Premier zapowiedział zwiększenie wydatków na ochro...
   Przewidziano: polityka | Oczekiwano: polityka
8. Apple przedstawiło nowy procesor M3 z architekturą...
   Przewidziano: technologia | Oczekiwano: technologia


In [53]:
# WARIANT 2: FEW-SHOT
print("\n--- WARIANT 2: FEW-SHOT ---")

SYSTEM_FEW = "Klasyfikuj teksty do jednej z kategorii: nauka, sport, polityka, technologia."

few_shot_examples = [
    {
        "role": "user",
        "content": "Tekst: \"Badacze z Harvardu opracowali nową szczepionkę przeciwko grypie.\""
    },
    {
        "role": "assistant", 
        "content": "{\"kategoria\":\"nauka\"}"
    },
    {
        "role": "user",
        "content": "Tekst: \"Cristiano Ronaldo zdobył bramkę w 90. minucie meczu.\""
    },
    {
        "role": "assistant",
        "content": "{\"kategoria\":\"sport\"}"
    },
    {
        "role": "user", 
        "content": "Tekst: \"Sejm uchwalił budżet państwa na następny rok.\""
    },
    {
        "role": "assistant",
        "content": "{\"kategoria\":\"polityka\"}"
    },
    {
        "role": "user",
        "content": "Tekst: \"Google zaprezentowało nowy algorytm uczenia maszynowego.\""
    },
    {
        "role": "assistant",
        "content": "{\"kategoria\":\"technologia\"}"
    }
]

few_shot_results = []

for i, text in enumerate(test_texts):
    try:
        messages = [{"role": "system", "content": SYSTEM_FEW}] + few_shot_examples + [
            {"role": "user", "content": f"Tekst: \"{text}\""}
        ]
        
        response = ollama.chat(
            model='gemma2:2b',
            messages=messages,
            format="json",
            options={"temperature": 0}
        )
        
        result = json.loads(response['message']['content'])
        predicted = result.get('kategoria', 'BŁĄD')
        few_shot_results.append(predicted)
        
        print(f"{i+1}. {text[:50]}...")
        print(f"   Przewidziano: {predicted} | Oczekiwano: {expected_categories[i]}")
        
    except Exception as e:
        print(f"Błąd: {e}")
        few_shot_results.append('BŁĄD')


--- WARIANT 2: FEW-SHOT ---
1. Nowy mikroskop pozwala obserwować pojedyncze atomy...
   Przewidziano: nauka | Oczekiwano: nauka
2. Robert Lewandowski strzelił hat-tricka w meczu prz...
   Przewidziano: sport | Oczekiwano: sport
3. Parlament przyjął ustawę o reformie systemu podatk...
   Przewidziano: polityka | Oczekiwano: polityka
4. Sztuczna inteligencja ChatGPT-4 osiągnęła przełomo...
   Przewidziano: nauka | Oczekiwano: technologia
5. Naukowcy z MIT odkryli nowy materiał przewodzący p...
   Przewidziano: nauka | Oczekiwano: nauka
6. Reprezentacja Polski awansowała do półfinału mistr...
   Przewidziano: sport | Oczekiwano: sport
7. Premier zapowiedział zwiększenie wydatków na ochro...
   Przewidziano: polityka | Oczekiwano: polityka
8. Apple przedstawiło nowy procesor M3 z architekturą...
   Przewidziano: technologia | Oczekiwano: technologia


In [54]:
# WARIANT 3: SEKCYJNY (RULES-INPUT-OUTPUT)  
print("\n--- WARIANT 3: SEKCYJNY ---")

def classify_sectional(text):
    prompt = f"""
<RULES>
Przypisz tekst dokładnie do jednej z 4 kategorii:
- nauka: badania, odkrycia, eksperymenty, publikacje naukowe
- sport: mecze, zawody, wyniki sportowe, zawodnicy
- polityka: wybory, ustawy, rząd, partie, decyzje polityczne  
- technologia: nowe urządzenia, oprogramowanie, IT, innowacje tech
</RULES>

<INPUT>
{text}
</INPUT>

<OUTPUT>
Zwróć JSON: {{"kategoria": "nauka/sport/polityka/technologia", "uzasadnienie": "krótkie uzasadnienie"}}
</OUTPUT>
"""

    response = ollama.chat(
        model='gemma2:2b',
        messages=[
            {"role": "system", "content": "Jesteś precyzyjnym klasyfikatorem. Postępuj zgodnie z instrukcjami w sekcjach."},
            {"role": "user", "content": prompt}
        ],
        format="json",
        options={"temperature": 0}
    )
    
    return response['message']['content']

sectional_results = []

for i, text in enumerate(test_texts):
    try:
        result_raw = classify_sectional(text)
        result = json.loads(result_raw)
        predicted = result.get('kategoria', 'BŁĄD')
        sectional_results.append(predicted)
        
        print(f"{i+1}. {text[:50]}...")
        print(f"   Przewidziano: {predicted} | Oczekiwano: {expected_categories[i]}")
        print(f"   Uzasadnienie: {result.get('uzasadnienie', 'Brak')}")
        
    except Exception as e:
        print(f"Błąd: {e}")
        sectional_results.append('BŁĄD')


--- WARIANT 3: SEKCYJNY ---
1. Nowy mikroskop pozwala obserwować pojedyncze atomy...
   Przewidziano: nauka | Oczekiwano: nauka
   Uzasadnienie: Tekst opisuje nowy mikroskop, który pozwala na obserwację pojedynczych atomy w czasie rzeczywistym. To dotyczy badań naukowych.
2. Robert Lewandowski strzelił hat-tricka w meczu prz...
   Przewidziano: sport | Oczekiwano: sport
   Uzasadnienie: Zawodnik strzelił hat-trick w meczu, co jest wydarzeniem sportowym.
3. Parlament przyjął ustawę o reformie systemu podatk...
   Przewidziano: polityka | Oczekiwano: polityka
   Uzasadnienie: W tekście opisano przyjęcie ustawy przez parlament, co jest elementem polityki.
4. Sztuczna inteligencja ChatGPT-4 osiągnęła przełomo...
   Przewidziano: nauka | Oczekiwano: technologia
   Uzasadnienie: Tekst opisuje osiągnięcia w dziedzinie sztucznej inteligencji, co jest tematem badań i naukowych odkryć.
5. Naukowcy z MIT odkryli nowy materiał przewodzący p...
   Przewidziano: nauka | Oczekiwano: nauka
   Uzasadn

In [55]:
# ANALIZA WYNIKÓW
print("\n=== ANALIZA PORÓWNAWCZA ===")

def calculate_accuracy(predicted, expected):
    correct = sum(1 for p, e in zip(predicted, expected) if p == e)
    total = len(expected)
    return correct / total if total > 0 else 0

accuracy_zero = calculate_accuracy(zero_shot_results, expected_categories)
accuracy_few = calculate_accuracy(few_shot_results, expected_categories)
accuracy_sectional = calculate_accuracy(sectional_results, expected_categories)

print(f"Dokładność Zero-shot: {accuracy_zero:.2%}")
print(f"Dokładność Few-shot: {accuracy_few:.2%}")
print(f"Dokładność Sekcyjna: {accuracy_sectional:.2%}")

# Tabela porównawcza
print("\n--- TABELA PORÓWNAWCZA ---")
print("Nr | Oczekiwane | Zero-shot | Few-shot | Sekcyjna")
print("-" * 50)

for i in range(len(test_texts)):
    expected = expected_categories[i]
    zero = zero_shot_results[i] 
    few = few_shot_results[i]
    sect = sectional_results[i]
    
    print(f"{i+1:2d} | {expected:10s} | {zero:9s} | {few:8s} | {sect:8s}")

# Analiza błędów
print("\n--- ANALIZA BŁĘDÓW ---")
categories = ["nauka", "sport", "polityka", "technologia"]

for strategy_name, results in [("Zero-shot", zero_shot_results), 
                               ("Few-shot", few_shot_results),
                               ("Sekcyjna", sectional_results)]:
    print(f"\n{strategy_name}:")
    errors = [(i, expected_categories[i], results[i]) 
              for i in range(len(results)) 
              if results[i] != expected_categories[i]]
    
    if errors:
        for idx, expected, predicted in errors:
            print(f"  Błąd {idx+1}: {expected} → {predicted}")
            print(f"    Tekst: {test_texts[idx][:60]}...")
    else:
        print("  Brak błędów!")

print(f"\n=== PODSUMOWANIE ===")
print(f"Najlepsza strategia: ", end="")
best_accuracy = max(accuracy_zero, accuracy_few, accuracy_sectional)
if best_accuracy == accuracy_sectional:
    print("Sekcyjna")
elif best_accuracy == accuracy_few:
    print("Few-shot")  
else:
    print("Zero-shot")


=== ANALIZA PORÓWNAWCZA ===
Dokładność Zero-shot: 75.00%
Dokładność Few-shot: 87.50%
Dokładność Sekcyjna: 75.00%

--- TABELA PORÓWNAWCZA ---
Nr | Oczekiwane | Zero-shot | Few-shot | Sekcyjna
--------------------------------------------------
 1 | nauka      | technologia | nauka    | nauka   
 2 | sport      | sport     | sport    | sport   
 3 | polityka   | polityka  | polityka | polityka
 4 | technologia | technologia | nauka    | nauka   
 5 | nauka      | technologia | nauka    | nauka   
 6 | sport      | sport     | sport    | sport   
 7 | polityka   | polityka  | polityka | polityka
 8 | technologia | technologia | technologia | nauka   

--- ANALIZA BŁĘDÓW ---

Zero-shot:
  Błąd 1: nauka → technologia
    Tekst: Nowy mikroskop pozwala obserwować pojedyncze atomy w czasie ...
  Błąd 5: nauka → technologia
    Tekst: Naukowcy z MIT odkryli nowy materiał przewodzący prąd w temp...

Few-shot:
  Błąd 4: technologia → nauka
    Tekst: Sztuczna inteligencja ChatGPT-4 osiągnęła prze

Zadanie 7.5 Streszczenie i parafraza

Generuj dwa streszczenia (krótkie i długie) oraz parafrazę krótkiego.

```python
@'
<ROLE>Tworzysz streszczenia i parafrazy. Odpowiadasz w JSON.</ROLE>
<TEXT>
Sztuczna inteligencja wspiera naukowców w analizie danych medycznych,
pomagając szybciej diagnozować choroby i tworzyć spersonalizowane terapie.
</TEXT>
<OUTPUT>
'@ | ollama run gemma2:2b
```

In [59]:
SYSTEM = "Jesteś ekspertem od streszczania i parafrazowania tekstu. Zwracasz wyłącznie poprawny JSON."

text = """
Rewolucja w dziedzinie sztucznej inteligencji przyspiesza w zawrotnym tempie. Nowe modele językowe, 
takie jak GPT-4 czy Claude, potrafią już nie tylko generować tekst, ale także rozumować, kodować 
i rozwiązywać złożone problemy. Firmy technologiczne inwestują miliardy dolarów w rozwój AI, 
przewidując, że będzie to kluczowa technologia następnej dekady.

Jednak rozwój sztucznej inteligencji niesie ze sobą również wyzwania. Eksperci ostrzegają przed 
ryzykiem utraty miejsc pracy w wielu sektorach, problemami z dezinformacją oraz kwestiami 
bezpieczeństwa cybernetycznego. Rządy na całym świecie pracują nad regulacjami, które miałyby 
kontrolować rozwój AI, zachowując równowagę między innowacją a bezpieczeństwem publicznym.

W sektorze edukacji AI już teraz zmienia sposób nauczania i uczenia się. Spersonalizowane systemy 
edukacyjne dostosowują się do indywidualnych potrzeb uczniów, podczas gdy nauczyciele korzystają 
z narzędzi AI do tworzenia materiałów dydaktycznych i oceniania prac. Przyszłość edukacji będzie 
prawdopodobnie opierać się na współpracy między człowiekiem a sztuczną inteligencją.
"""

print(f"ORYGINALNY TEKST ({len(text.split())} słów):")
print(text.strip())
print("\n" + "="*60 + "\n")

# PROMPT W SCHEMACIE SEKCYJNYM
prompt = f"""
<ROLE>
Jesteś specjalistą od przetwarzania tekstu. Tworzysz wysokiej jakości streszczenia i parafrazy.
</ROLE>

<RULES>
1. Krótkie streszczenie: maksymalnie 25 słów, zachowaj najważniejsze informacje
2. Długie streszczenie: maksymalnie 70 słów, uwzględnij kluczowe szczegóły
3. Parafraza: przepisz krótkie streszczenie innymi słowami, zachowaj sens
4. Zwróć wynik w formacie JSON
</RULES>

<INPUT>
{text}
</INPUT>

<OUTPUT>
{{
  "short_summary": "krótkie streszczenie (max 25 słów)",
  "long_summary": "długie streszczenie (max 70 słów)", 
  "paraphrase": "parafraza krótkiego streszczenia"
}}
</OUTPUT>
"""

response = ollama.chat(
    model='gemma2:2b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": prompt}
    ],
    format="json",
    options={"temperature": 0}
)

result = json.loads(response['message']['content'])

print("=== WYNIKI STRESZCZANIA I PARAFRAZOWANIA ===\n")

print("KRÓTKIE STRESZCZENIE:")
print(f"→ {result['short_summary']}")
print(f"  Długość: {len(result['short_summary'].split())} słów\n")

print("DŁUGIE STRESZCZENIE:")
print(f"→ {result['long_summary']}")
print(f"  Długość: {len(result['long_summary'].split())} słów\n")

print("PARAFRAZA KRÓTKIEGO STRESZCZENIA:")
print(f"→ {result['paraphrase']}")
print(f"  Długość: {len(result['paraphrase'].split())} słów\n")

ORYGINALNY TEKST (140 słów):
Rewolucja w dziedzinie sztucznej inteligencji przyspiesza w zawrotnym tempie. Nowe modele językowe, 
takie jak GPT-4 czy Claude, potrafią już nie tylko generować tekst, ale także rozumować, kodować 
i rozwiązywać złożone problemy. Firmy technologiczne inwestują miliardy dolarów w rozwój AI, 
przewidując, że będzie to kluczowa technologia następnej dekady.

Jednak rozwój sztucznej inteligencji niesie ze sobą również wyzwania. Eksperci ostrzegają przed 
ryzykiem utraty miejsc pracy w wielu sektorach, problemami z dezinformacją oraz kwestiami 
bezpieczeństwa cybernetycznego. Rządy na całym świecie pracują nad regulacjami, które miałyby 
kontrolować rozwój AI, zachowując równowagę między innowacją a bezpieczeństwem publicznym.

W sektorze edukacji AI już teraz zmienia sposób nauczania i uczenia się. Spersonalizowane systemy 
edukacyjne dostosowują się do indywidualnych potrzeb uczniów, podczas gdy nauczyciele korzystają 
z narzędzi AI do tworzenia materiałów dy

In [60]:
# TEST Z DRUGIM MODELEM
print("=== PORÓWNANIE Z INNYM MODELEM ===\n")

response2 = ollama.chat(
    model='llama3.1:8b',
    messages=[
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": prompt}
    ],
    format="json",
    options={"temperature": 0}
)

result2 = json.loads(response2['message']['content'])

print("MODEL: llama3.1:8b")
print(f"Krótkie: {result2['short_summary']}")
print(f"Długie: {result2['long_summary']}")
print(f"Parafraza: {result2['paraphrase']}")

=== PORÓWNANIE Z INNYM MODELEM ===

MODEL: llama3.1:8b
Krótkie: Rewolucja w sztucznej inteligencji przyspiesza, nowe modele językowe potrafią rozumować i kodować.
Długie: Rozwój sztucznej inteligencji jest szybki, firmy inwestują miliardy dolarów. Eksperci ostrzegają przed ryzykiem utraty miejsc pracy i problemami z dezinformacją. Rządy pracują nad regulacjami.
Parafraza: Sztuczna inteligencja przyspiesza, nowe technologie potrafią rozumować i kodować, ale niesie również ryzyko utraty miejsc pracy i problemów z dezinformacją.


In [ ]:
CONTEXT = """
Komputery kwantowe wykorzystują zjawiska mechaniki kwantowej, takie jak superpozycja i splątanie, 
do przetwarzania informacji. W przeciwieństwie do klasycznych bitów, które mogą być w stanie 0 lub 1, 
kubity (bity kwantowe) mogą znajdować się w superpozycji obu stanów jednocześnie.

IBM zaprezentowało w 2023 roku procesor kwantowy Eagle z 433 kubitami, który jest jednym z najbardziej 
zaawansowanych systemów komercyjnych. Google z kolei twierdzi, że osiągnęło "przewagę kwantową" 
ze swoim procesorem Sycamore w 2019 roku, wykonując obliczenia, które zajęłyby klasycznemu 
superkomputerowi tysiące lat.

Główne zastosowania komputerów kwantowych obejmują kryptografię, optymalizację, symulacje molekularne 
oraz uczenie maszynowe. Jednak technologia ta wciąż boryka się z problemami dekoherencji kwantowej 
i wysokimi wymaganiami dotyczącymi temperatury - większość systemów wymaga chłodzenia do temperatur 
bliskich zeru absolutnemu.

Eksperci przewidują, że praktyczne zastosowania komputerów kwantowych w przemyśle staną się powszechne 
dopiero za 10-15 lat, gdy technologia osiągnie większą stabilność i skalowalność.
"""

questions_in_context = [
    "Ile kubitów ma procesor Eagle firmy IBM?",
    "Jakie są główne zastosowania komputerów kwantowych?",
    "Kiedy Google osiągnęło przewagę kwantową?",
    "Jakie są główne problemy komputerów kwantowych?"
]

questions_out_of_context = [
    "Jaka jest cena procesora Eagle?",
    "Kto jest CEO firmy IBM?",
    "Gdzie produkowane są komputery kwantowe?",
    "Ile kosztuje budowa komputera kwantowego?"
]

print(f"KONTEKST ({len(CONTEXT.split())} słów):")
print(CONTEXT.strip())
print("\n" + "="*70 + "\n")

# WARIANT 1: BEZ ZASAD
def qa_without_rules(context, question, model='gemma2:2b'):
    """QA bez specjalnych zasad - podstawowy prompt"""
    
    prompt = f"""
    Na podstawie poniższego tekstu odpowiedz na pytanie.

    Tekst:
    {context}

    Pytanie: {question}
    """
    
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        options={"temperature": 0}
    )
    
    return response['message']['content']

# WARIANT 2: ZE ŚCISŁYMI ZASADAMI
def qa_with_strict_rules(context, question, model='gemma2:2b'):
    """QA ze ścisłymi zasadami dotyczącymi odpowiedzi"""
    
    SYSTEM = """
    Jesteś precyzyjnym asystentem QA. Odpowiadaj WYŁĄCZNIE na podstawie dostarczonego kontekstu.
    ZASADY:
    1. Jeśli odpowiedź nie znajduje się w kontekście, odpowiedz: "Brak wystarczających informacji"
    2. Cytuj fragment źródłowy w nawiasach [] gdy to możliwe
    3. Bądź zwięzły i konkretny
    4. NIE dodawaj informacji spoza kontekstu
    """
    
    prompt = f"""
    <KONTEKST>
    {context}
    </KONTEKST>

    <PYTANIE>
    {question}
    </PYTANIE>

    Odpowiedź:
    """
    
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt}
        ],
        options={"temperature": 0}
    )
    
    return response['message']['content']

# WARIANT 3: FEW-SHOT Z PRZYKŁADAMI
def qa_few_shot(context, question, model='gemma2:2b'):
    """QA z przykładami few-shot"""
    
    SYSTEM = "Odpowiadaj na pytania na podstawie kontekstu. Jeśli informacji brak, napisz 'Brak wystarczających informacji'."
    
    examples = [
        {
            "role": "user",
            "content": """Kontekst: Apple wydało iPhone 15 we wrześniu 2023 z nowym procesorem A17 Pro. Pytanie: Kiedy wydano iPhone 15?"""
        },
        {
            "role": "assistant", 
            "content": "iPhone 15 wydano we wrześniu 2023 [Apple wydało iPhone 15 we wrześniu 2023]."
        },
        {
            "role": "user",
            "content": """Kontekst: Apple wydało iPhone 15 we wrześniu 2023 z nowym procesorem A17 Pro. Pytanie: Jaka jest cena iPhone 15?"""
        },
        {
            "role": "assistant",
            "content": "Brak wystarczających informacji"
        }
    ]
    
    messages = [{"role": "system", "content": SYSTEM}] + examples + [
        {
            "role": "user", 
            "content": f"Kontekst: {context}\nPytanie: {question}"
        }
    ]
    
    response = ollama.chat(
        model=model,
        messages=messages,
        options={"temperature": 0}
    )
    
    return response['message']['content']


In [ ]:
# TESTOWANIE WARIANTÓW
print("=== PORÓWNANIE WARIANTÓW PROMPTÓW ===\n")

strategies = [
    ("Bez zasad", qa_without_rules),
    ("Ścisłe zasady", qa_with_strict_rules), 
    ("Few-shot", qa_few_shot)
]

# Test 1: Pytania w kontekście
print("--- PYTANIA W KONTEKŚCIE ---")
for question in questions_in_context:
    print(f"\nPYTANIE: {question}")
    print("-" * 50)
    
    for strategy_name, strategy_func in strategies:
        try:
            answer = strategy_func(CONTEXT, question)
            print(f"{strategy_name:12}: {answer}")
        except Exception as e:
            print(f"{strategy_name:12}: BŁĄD - {e}")

print("\n\n--- PYTANIA POZA KONTEKSTEM ---")
for question in questions_out_of_context:
    print(f"\nPYTANIE: {question}")
    print("-" * 50)
    
    for strategy_name, strategy_func in strategies:
        try:
            answer = strategy_func(CONTEXT, question)
            print(f"{strategy_name:12}: {answer}")
        except Exception as e:
            print(f"{strategy_name:12}: BŁĄD - {e}")

# ANALIZA WYNIKÓW
print("\n\n=== ANALIZA SKUTECZNOŚCI ===")

def evaluate_answer(answer, should_have_info=True):
    """Prosta ewaluacja odpowiedzi"""
    no_info_phrases = ["brak wystarczających", "nie ma informacji", "brak informacji", 
                      "nie można odpowiedzieć", "nieznane", "brak danych"]
    
    has_no_info_response = any(phrase in answer.lower() for phrase in no_info_phrases)
    
    if should_have_info:
        return "POPRAWNE" if not has_no_info_response else "BŁĘDNE (fałszywy negatyw)"
    else:
        return "POPRAWNE" if has_no_info_response else "BŁĘDNE (halucynacja)"

# Testowanie na przykładach
test_cases = [
    ("Ile kubitów ma procesor Eagle?", True),
    ("Jaka jest cena procesora Eagle?", False),
    ("Kiedy Google osiągnęło przewagę kwantową?", True),  
    ("Kto jest CEO firmy IBM?", False)
]

print("\nWYNIKI EWALUACJI:")
print("Strategia     | W kontekście | Poza kontekstem | Ogólna ocena")
print("-" * 65)

for strategy_name, strategy_func in strategies:
    correct_in = 0
    correct_out = 0
    
    # Test pytań w kontekście
    for question, should_have_info in [tc for tc in test_cases if tc[1]]:
        answer = strategy_func(CONTEXT, question)
        if "POPRAWNE" in evaluate_answer(answer, should_have_info):
            correct_in += 1
    
    # Test pytań poza kontekstem        
    for question, should_have_info in [tc for tc in test_cases if not tc[1]]:
        answer = strategy_func(CONTEXT, question)
        if "POPRAWNE" in evaluate_answer(answer, should_have_info):
            correct_out += 1
            
    total_score = (correct_in + correct_out) / 4 * 100
    
    print(f"{strategy_name:12} | {correct_in}/2          | {correct_out}/2            | {total_score:.0f}%")

print(f"\n=== WNIOSKI ===")
print("1. Wariant 'Ścisłe zasady' najlepiej filtruje odpowiedzi")
print("2. 'Bez zasad' ma tendencję do halucynacji")  
print("3. 'Few-shot' daje stabilne rezultaty dzięki przykładom")
print("4. Cytowanie źródeł [] zwiększa wiarygodność odpowiedzi")